In [ ]:
#pip install pyreadr

In [35]:
import pandas as pd
import pyreadr

# Data Cleaning

## 0. Delineation File

In [178]:
delineation_df = pd.read_excel('./data/delineation_file.xls')
delineation_df = delineation_df.iloc[1:1918]
delineation_df = delineation_df.rename(columns=delineation_df.iloc[0])
delineation_df = delineation_df.drop(1)
delineation_df['FIPS State Code'] = delineation_df['FIPS State Code'].astype(int)
delineation_df['CBSA Code'] = delineation_df['CBSA Code'].astype(int)
delineation_df = delineation_df[['CBSA Code', 'State Name', 'FIPS State Code']]
delineation_df

,CBSA Code,State Name,FIPS State Code
2,10100,South Dakota,46
3,10100,South Dakota,46
4,10140,Washington,53
5,10180,Texas,48
6,10180,Texas,48
...,...,...,...
1913,49700,California,6
1914,49700,California,6
1915,49740,Arizona,4
1916,49780,Ohio,39


In [128]:
code = pyreadr.read_r('./data/code.Rds')

In [174]:
code_df = code[None]
code_df['cityID'] = code_df['cityID'].astype(int)
code_df = code_df.drop(columns=['NAME', 'ORI', 'countyID'])
code_df

,cityID,cbsa
0,100124,20020
1,100460,13820
2,100484,NaN
3,1005690,42580
4,1005820,42580
...,...,...
22721,999999,25540
22722,999999,35980
22723,999999,48740
22724,999999,14860


In [220]:
code_merged = pd.merge(delineation_df, code_df, left_on='CBSA Code', right_on='cbsa', how='right')
code_merged.drop_duplicates()

,CBSA Code,State Name,FIPS State Code,cityID,cbsa
0,20020,Alabama,1.0,100124,20020
3,13820,Alabama,1.0,100460,13820
9,NaN,NaN,NaN,100484,NaN
10,NaN,NaN,NaN,1005690,42580
11,NaN,NaN,NaN,1005820,42580
...,...,...,...,...,...
73274,25540,Connecticut,9.0,999999,25540
73277,35980,Connecticut,9.0,999999,35980
73278,NaN,NaN,NaN,999999,48740
73279,14860,Connecticut,9.0,999999,14860


## 1. Crime Data

In [182]:
crime_df = pd.read_csv('./data/crime.csv')

In [216]:
crime = crime_df[['cbsa', 'STATE', 'ORI', 'cityID', 'YEAR', 'MONTH', 'OFFENSE', 'OCCUR', 'POP']]
#crime_df = crime_df[crime_df['POP'] >= 3000]
for index, row in crime.iterrows():
    if crime['STATE'][index] == 'south' and crime['ORI'][index][0:2] == 'SC':
        crime['STATE'][index] = 'south carolina'
    if crime['STATE'][index] == 'south' and crime['ORI'][index][0:2] == 'SD':
        crime['STATE'][index] = 'south dakota'
    if crime['STATE'][index] == 'north' and crime['ORI'][index][0:2] == 'NC':
        crime['STATE'][index] = 'north carolina'
    if crime['STATE'][index] == 'north' and crime['ORI'][index][0:2] == 'ND':
        crime['STATE'][index] = 'north dakota'
    if crime['STATE'][index] == 'west':
        crime['STATE'][index] = 'west virginia'
    if crime['STATE'][index] == 'district':
        crime['STATE'][index] = 'district of columbia'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NH':
        crime['STATE'][index] = 'new hampshire'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NJ':
        crime['STATE'][index] = 'new jersey'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NM':
        crime['STATE'][index] = 'new mexico'
    if crime['STATE'][index] == 'new' and crime['ORI'][index][0:2] == 'NY':
        crime['STATE'][index] = 'new york'
    if crime['STATE'][index] == 'rhode':
        crime['STATE'][index] = 'rhode island'
        

delineation_df_small = delineation_df[['FIPS State Code', 'State Name']]
delineation_df_small['State Name'] = delineation_df_small['State Name'].str.lower()
crime = pd.merge(crime, delineation_df_small, left_on='STATE',right_on='State Name', how='left')

lost_state = crime[crime['FIPS State Code'].isnull()]
lost_state_list = set(lost_state['STATE'].to_list())
print(lost_state_list)
crime = crime.drop(columns=['STATE', 'cityID', 'POP', 'OFFENSE'])
crime

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/2064130986.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime['STATE'][index] = 'new jersey'
/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/2064130986.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime['STATE'][index] = 'new mexico'
/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/2064130986.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

{'guam'}


,cbsa,STATE,ORI,cityID,YEAR,MONTH,OFFENSE,OCCUR,POP,FIPS State Code,State Name
0,10700.0,alabama,AL05004,107912.0,2015,5,20,2,9740,1.0,alabama
1,10700.0,alabama,AL05004,107912.0,2015,5,20,2,9740,1.0,alabama
2,10700.0,alabama,AL05004,107912.0,2015,5,20,2,9740,1.0,alabama
3,10700.0,alabama,AL05004,107912.0,2015,5,20,2,9740,1.0,alabama
4,10700.0,alabama,AL05004,107912.0,2015,5,20,2,9740,1.0,alabama
...,...,...,...,...,...,...,...,...,...,...,...
6844051,49780.0,ohio,OH06001,3988084.0,2014,7,2,2,25415,39.0,ohio
6844052,49780.0,ohio,OH06001,3988084.0,2014,7,2,2,25415,39.0,ohio
6844053,49780.0,ohio,OH06001,3988084.0,2014,7,2,2,25415,39.0,ohio
6844054,49780.0,ohio,OH06001,3988084.0,2014,7,2,2,25415,39.0,ohio


In [230]:
crime_agg = crime.groupby(['MONTH','YEAR','FIPS State Code']).size().reset_index(name='crime_counts')
crime_agg = crime_agg.rename(columns={'MONTH': 'month'})
crime_agg['month'] = crime_agg['month'].astype(int)
crime_agg['YEAR'] = crime_agg['YEAR'].astype(int)
crime_agg = crime_agg.dropna(subset=['FIPS State Code'])
crime_agg

,month,YEAR,FIPS State Code,crime_counts
0,1,2012,2.0,45
1,1,2012,4.0,648
2,1,2012,5.0,656
3,1,2012,6.0,13860
4,1,2012,8.0,1450
...,...,...,...,...
2301,12,2015,51.0,8900
2302,12,2015,53.0,1932
2303,12,2015,54.0,899
2304,12,2015,55.0,7920


## 2. Alcohol Data

In [254]:
alcohol_df = pd.read_csv('./data/alcohol.csv')
alcohol_df['alcohol_per_month'] = round(alcohol_df['alcohol']/12, 2)
alcohol_df = alcohol_df.fillna(0)
alcohol_df = pd.merge(alcohol_df, delineation_df, left_on='cbsa', right_on='CBSA Code', how='left')
alcohol_df = alcohol_df.dropna(subset=['FIPS State Code'])
alcohol_df = alcohol_df.drop(columns=['State Name', 'cbsa', 'CBSA Code'])
alcohol_df = alcohol_df.groupby(['YEAR','FIPS State Code']).size().reset_index(name='alcohol_counts')
alcohol_df

,YEAR,FIPS State Code,alcohol_counts
0,2012,1.0,44
1,2012,2.0,5
2,2012,4.0,11
3,2012,5.0,41
4,2012,6.0,45
...,...,...,...
250,2016,51.0,89
251,2016,53.0,28
252,2016,54.0,30
253,2016,55.0,40


## 3. Twitter Data

In [265]:
twitter_df = pd.read_csv('./data/twitter_example.csv', encoding = "ISO-8859-1")
# only select useful columns
twitter_df = twitter_df[['id_str', 'UScounty', 'UScity', 'USstate', 'day', 'month', 'YEAR', 'user_id_str', 'lang',
                         'retweet_count', 'statuses_count', 'followers_count', 'favourites_count', 'friends_count']]
# drop missing value
twitter_df = twitter_df.dropna(subset = ['UScity', 'USstate'], how='all')
# add cbsa code
twitter_df = twitter_df.fillna(0)
twitter_df['UScity'] = twitter_df['UScity'].astype(int)
twitter_df['USstate'] = twitter_df['USstate'].astype(int)
twitter_df = pd.merge(twitter_df, code_df, left_on='UScity', right_on='cityID', how='left') #4948
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])

delineation_df_small = delineation_df[['CBSA Code', 'FIPS State Code']]
twitter_df = pd.merge(twitter_df, delineation_df_small, left_on='cbsa', right_on='CBSA Code', how='left')
twitter_df = twitter_df.drop_duplicates(subset=['id_str'])
twitter_df = twitter_df.drop(columns=['cityID', 'cbsa', 'CBSA Code'])
twitter_df = twitter_df.fillna(0)
for index, row in twitter_df.iterrows():
    if twitter_df['USstate'][index]==0:
        code = twitter_df['FIPS State Code'][index]
        twitter_df['USstate'][index] = code
twitter_df = twitter_df.drop(columns=['lang','FIPS State Code', 'UScounty', 'UScity', 'day', 'user_id_str', 'id_str'])
twitter_df = twitter_df[twitter_df['USstate']!=0]
twitter_df #4590

/var/folders/3h/x2xrdjwn22bbdw5d98hd13w80000gn/T/ipykernel_44595/2567128739.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_df['USstate'][index] = code


,USstate,month,YEAR,retweet_count,statuses_count,followers_count,favourites_count,friends_count,State Name
0,55,1,2013,0,1609,228,847,222,Wisconsin
4,25,1,2013,0,33,0,0,0,Massachusetts
11,36,1,2013,1,39932,1015,16,1032,New York
14,12,1,2013,0,2743,108,276,98,0
15,51,1,2013,2,13105,462,340,70,0
...,...,...,...,...,...,...,...,...,...
26865,12,7,2015,0,1304,2083,0,2263,Florida
26870,53,7,2015,0,14286,255,4243,220,0
26871,24,7,2015,65,101939,1706,2350,1697,Maryland
26878,47,7,2015,53,5548,392,1281,1007,Tennessee


In [243]:
# group by month, year, USstate
twitter_agg1 = twitter_df.groupby(['month','YEAR','USstate']).size().reset_index(name='tweets_counts')
twitter_agg2 = twitter_df.groupby(['month','YEAR','USstate']).sum()
twitter_agg = pd.merge(twitter_agg1, twitter_agg2, on=['month','YEAR','USstate'])
twitter_agg = twitter_agg.rename(columns={'USstate': 'FIPS State Code'})
twitter_agg

,month,YEAR,FIPS State Code,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count
0,1,2013,1,4,9,43398,2647,4193,2346
1,1,2013,4,2,3,24673,714,459,724
2,1,2013,5,2,0,48629,490,107,534
3,1,2013,6,25,334,238686,11422,39843,10282
4,1,2013,8,3,0,2056,190,242,916
...,...,...,...,...,...,...,...,...,...
1070,12,2015,40,2,3,108411,1693,33833,1184
1071,12,2015,41,1,6242,36147,529,7747,283
1072,12,2015,42,2,1159,9095,994,5039,873
1073,12,2015,48,21,45015,877925,34591,181835,29483


# Merge All Data

In [290]:
# alcohol and twitter
al_twitter = pd.merge(twitter_agg, alcohol_df, on=['FIPS State Code','YEAR'], how='left')

# add crime
merged_df = pd.merge(al_twitter, crime_agg, on=['FIPS State Code', 'YEAR', 'month'], how='left')
merged_df = merged_df.fillna(0)
merged_df

,month,YEAR,FIPS State Code,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count,alcohol_counts,crime_counts
0,1,2013,1,4,9,43398,2647,4193,2346,44,0.0
1,1,2013,4,2,3,24673,714,459,724,11,552.0
2,1,2013,5,2,0,48629,490,107,534,41,697.0
3,1,2013,6,25,334,238686,11422,39843,10282,45,8640.0
4,1,2013,8,3,0,2056,190,242,916,29,1914.0
...,...,...,...,...,...,...,...,...,...,...,...
1070,12,2015,40,2,3,108411,1693,33833,1184,37,1554.0
1071,12,2015,41,1,6242,36147,529,7747,283,26,1040.0
1072,12,2015,42,2,1159,9095,994,5039,873,54,12474.0
1073,12,2015,48,21,45015,877925,34591,181835,29483,128,47553.0


In [291]:
state_code = pd.read_csv('./data/us-state-ansi-fips.csv')
state_code = state_code[['stname', ' st']]
merged_df = pd.merge(merged_df, state_code, left_on='FIPS State Code', right_on=' st', how='left')
merged_df = merged_df.drop(columns=[' st'])
merged_df

,month,YEAR,FIPS State Code,tweets_counts,retweet_count,statuses_count,followers_count,favourites_count,friends_count,alcohol_counts,crime_counts,stname
0,1,2013,1,4,9,43398,2647,4193,2346,44,0.0,Alabama
1,1,2013,4,2,3,24673,714,459,724,11,552.0,Arizona
2,1,2013,5,2,0,48629,490,107,534,41,697.0,Arkansas
3,1,2013,6,25,334,238686,11422,39843,10282,45,8640.0,California
4,1,2013,8,3,0,2056,190,242,916,29,1914.0,Colorado
...,...,...,...,...,...,...,...,...,...,...,...,...
1070,12,2015,40,2,3,108411,1693,33833,1184,37,1554.0,Oklahoma
1071,12,2015,41,1,6242,36147,529,7747,283,26,1040.0,Oregon
1072,12,2015,42,2,1159,9095,994,5039,873,54,12474.0,Pennsylvania
1073,12,2015,48,21,45015,877925,34591,181835,29483,128,47553.0,Texas


In [292]:
merged_df.to_csv('./data/merged_data.csv')